In [ ]:
!pip install torch torchvision matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from PIL import Image
import os


# Resize Dataset Images

In [ ]:
def resize_images(folder):
    for file in os.listdir(folder):
        path = os.path.join(folder, file)
        img = Image.open(path).convert('RGB')
        img = img.resize((224, 224))
        img.save(path)

resize_images("/content/drive/MyDrive/ImageClassification/melon_dataset/watermelon")
resize_images("/content/drive/MyDrive/ImageClassification/melon_dataset/cantaloupe")

# Splite Dataset

In [ ]:
import os, shutil
from sklearn.model_selection import train_test_split

def split_data(source_dir, target_dir, test_size=0.2, val_size=0.1):
    for category in os.listdir(source_dir):
        files = os.listdir(os.path.join(source_dir, category))
        train_files, test_files = train_test_split(files, test_size=test_size)
        train_files, val_files = train_test_split(train_files, test_size=val_size)

        for subset, data in zip(['train', 'val', 'test'], [train_files, val_files, test_files]):
            subset_dir = os.path.join(target_dir, subset, category)
            os.makedirs(subset_dir, exist_ok=True)
            for file in data:
                src = os.path.join(source_dir, category, file)
                dst = os.path.join(subset_dir, file)
                shutil.copy(src, dst)

split_data('/content/drive/MyDrive/ImageClassification/melon_dataset/', '/content/melon_dataset_split')


#Fine-Tuning ResNet50 with Pytorch


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models
import matplotlib.pyplot as plt
import time, copy, os




data_dir = "/content/melon_dataset_split"
num_classes = 2
batch_size = 16
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

device(type='cuda')

## Data Augmentation & Normalization

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}


# Load Dataset

In [ ]:
image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
    for x in ['train', 'val']
}
dataloaders = {
    x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True)
    for x in ['train', 'val']
}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

#  Load ResNet50 + replace last layer

In [ ]:
model = models.resnet50(pretrained=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 184MB/s]


# Freeze previous layers

In [ ]:
for param in model.parameters():
    param.requires_grad = False


# Change the last layer for 2-class classification

In [ ]:
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)


# Loss and Optimizer only for new layer

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)


# Training

In [15]:
def train_model(model, criterion, optimizer, num_epochs=10):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")

        for phase in ['train', 'val']:
            model.train() if phase == 'train' else model.eval()
            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            #Save best model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    print(f"Best val Acc: {best_acc:.4f}")
    model.load_state_dict(best_model_wts)
    return model


# Start Traning model
model = train_model(model, criterion, optimizer, num_epochs=num_epochs)

#Save finall model
torch.save(model.state_dict(), "resnet50_melon_classifier.pth")

Epoch 1/10
train Loss: 0.6663 Acc: 0.5741
val Loss: 0.4416 Acc: 0.7333
Epoch 2/10
train Loss: 0.4252 Acc: 0.8099
val Loss: 0.3005 Acc: 0.8667
Epoch 3/10
train Loss: 0.2966 Acc: 0.8783
val Loss: 0.2087 Acc: 0.9333
Epoch 4/10
train Loss: 0.1937 Acc: 0.9772
val Loss: 0.1606 Acc: 1.0000
Epoch 5/10
train Loss: 0.1810 Acc: 0.9544
val Loss: 0.1431 Acc: 1.0000
Epoch 6/10
train Loss: 0.1543 Acc: 0.9544
val Loss: 0.1447 Acc: 0.9667
Epoch 7/10
train Loss: 0.1789 Acc: 0.9392
val Loss: 0.2060 Acc: 0.9000
Epoch 8/10
train Loss: 0.1456 Acc: 0.9582
val Loss: 0.0970 Acc: 1.0000
Epoch 9/10
train Loss: 0.1293 Acc: 0.9620
val Loss: 0.1026 Acc: 0.9667
Epoch 10/10
train Loss: 0.1206 Acc: 0.9810
val Loss: 0.1138 Acc: 0.9667
Best val Acc: 1.0000
